# Making HI Cubes

Back in "Feedback_dominated_convert_binary_to_fits", I didn't make HI cubes. This is just a quick code to remedy that. 

nhi = nhtot - 2*nh2 - nhp

In [1]:
ls *.fits

nCO_x1470y1400z1400.fits         tdust_fits_x1470y1400z1400.fits
nH2_x1470y1400z1400.fits         temp_fits_x1470y1400z1400.fits
nhtot_x1470y1400z1400.fits


In [2]:
################################################
# 
# Imports
#
################################################

import numpy as np 
from astropy.io import fits 
import glob

# Loop

ulength=3.0856e+20 #internal unit length in centimeters
umass=1.9910000e+33 #internal unit mass in grams
uvel=1.0e+5 #internal unit velocity in cm/s
udensity=umass/ulength**3 #internal unit density in g/cm^-3
utime=3.0856000e+15 #internal unit time in seconds 
mp=1.67e-24 #mass of proton
dusttogas= 0.01 #dust to gas mass ratio
n = 500 #size of grids is 500 x 500 x 500 voxels, where each voxel = 1 pc in length

fn_density = glob.glob("density_grid*")
fn_xH2     = glob.glob("xH2_grid*")    
fn_xHP     = glob.glob("xHP_grid*")

for file_name in fn_density: 
    
    last_15 = file_name[-15:]
    print(last_15)
    fdensity = open(file_name, "r") # r is the default 
    
    # Right now the file is still binary, but we want it to be
    # an np array 
    
    # read in binary file to an array
    d = np.fromfile(fdensity, dtype=np.float32)
    d = np.delete(d, [0,1,2]) #delete first three entrees (which are not data)
    
    # reshape the 1D array to 3d (n x n x n)
    d = d.reshape([n,n,n]) # n = 500
    
    # convert from AREPO units to actual density (g/cm^3)
    rhogas = d*udensity
    
    # convert from total gas density to nubmber density of hydrogen nuclei 
    # the factor is 1.4 (mean molecular weight of H)
    ## mean molecular weight = average mass in terms of hydrogen masses
    ## mean molecular weight^-1 = 1/(1+4x_he), x_he = 0.1m 
    nhtot = rhogas/(1.4 * mp)
    ##############################
    ## H2 Abundance
    ##############################
    for file_name in fn_xH2: 
        
        if(file_name[-15:] == last_15):
            
            f_H2 = open(file_name,'r')
    
            d = np.fromfile(f_H2, dtype=np.float32)
            d = np.delete(d,[0,1,2])
            d = d.reshape([n,n,n])
            
            nh2 = d*nhtot
            break
        
    ##############################
    ## H+ Abundance
    ##############################
    for file_name in fn_xHP: 
    
        if(file_name[-15:] == last_15):
            
            f_HP = open(file_name,'r')
    
            d = np.fromfile(f_HP, dtype=np.float32)
            d = np.delete(d,[0,1,2])
            d = d.reshape([n,n,n])
        
            nhp = d*nhtot
            break
            
    nh1 = nhtot - 2*nh2 - nhp
    hdu = fits.PrimaryHDU(nh1)
    file_name = 'nH1_' + last_15 + '.fits'
    hdu.writeto(file_name,overwrite=True)
    
    

x1470y1400z1400


NameError: name 'nhp' is not defined